In [32]:
import pandas as pd
import numpy as np
import itertools
from xml.etree.ElementTree import ElementTree, Element, SubElement, tostring
import datetime

In [33]:
%matplotlib inline
pd.options.display.float_format = '{:,.2f}'.format

In [34]:
def datetime_to_secs(time):  
    if type(time) is datetime.time:
        secs = time.hour * 3600 + time.minute * 60 + time.second 
        return secs
    else:
        print('Provide a datetime.time var')

In [35]:
def randNormal_time(time, dev):
    mean = datetime_to_secs(datetime.datetime.strptime(time, '%H:%M').time())
    std = datetime_to_secs(datetime.datetime.strptime(dev, '%H:%M').time())
    # Draw from a normal distr
    time_secs = np.random.normal(loc=mean, scale=std)
    rand_time = str(datetime.timedelta(seconds=time_secs)).split('.')[0]
    return rand_time

In [36]:
class Agent(object):                   
    """ Agent class """
    def __init__(self, **kwargs):
        prop_defaults = {
            'id': 'non_defined'
        }

        for (prop, default) in prop_defaults.items():
            setattr(self, prop, kwargs.get(prop, default))

        self.plans = list()
        

In [37]:
class Plan(object):
    """ Plans class """
    def __init__(self, **kwargs):
        prop_defaults = {
            'selected': 'no'
        }

        for (prop, default) in prop_defaults.items():
            setattr(self, prop, kwargs.get(prop, default))

        self.activities = list()


In [38]:
class Activity(object):
    """ Activities class """    
    def __init__(self, **kwargs):
        prop_defaults = {
            'type': 'undefined',
            'link': None,
            'x': None,
            'y': None,
            'end_time': None
        }

        for (prop, default) in prop_defaults.items():
            setattr(self, prop, kwargs.get(prop, default))
        
        self.legs = list()
        

In [39]:
class Leg(object):
    """ Plans class """
    def __init__(self, **kwargs):
        prop_defaults = {
            'include_routes': False,
            'mode': 'car'
        }

        for (prop, default) in prop_defaults.items():
            setattr(self, prop, kwargs.get(prop, default))

        self.routes = list()


In [40]:
class Route(object):
    """ Plans class """
    def __init__(self, **kwargs):
        prop_defaults = {
            'type': 'links'
        }

        for (prop, default) in prop_defaults.items():
            setattr(self, prop, kwargs.get(prop, default))

        self.value = None

In [41]:
def write_xml_elem(c, root, d):
    for attr, v in c.__dict__.items():
        if isinstance(v, list):
            for e in v:
                write_xml_elem(e, root, d)
        else:
            if v:
                class_name = c.__class__.__name__
                elem = d[class_name]
                elem.set(attr, str(v))
                d[class_name] = elem
            

In [42]:
def build_pop_xml(pop):

    root = Element('population')
   
    for person in pop:
        xml_person = SubElement(root, 'person')
        xml_plan = SubElement(xml_person, 'plan')
        xml_activity = SubElement(xml_plan, 'act')
        xml_leg = SubElement(xml_plan, 'leg')
        xml_route = SubElement(xml_leg, 'route')

        d = {'Agent': xml_person,
             'Plan': xml_plan,
             'Activity': xml_activity,
             'Leg': xml_leg,
             'Route': xml_route}

        write_xml_elem(person, root, d)
    return root


# Inputs

## Zones

In [43]:
zone_centroids = pd.read_csv('../Network/Zones/zones_centroids_EPSG6312.csv', index_col=3)
zone_centroids = zone_centroids[['x', 'y']]
zone_centroids.head()

,x,y
NO,,
1,"532,581.66","3,892,173.91"
2,"532,426.63","3,892,226.77"
3,"532,114.61","3,892,154.10"
4,"532,030.41","3,892,609.90"
5,"531,746.95","3,892,600.84"


## Demand

In [44]:
mats = pd.read_csv('../Demand/matrices_demand.txt', delimiter=';', index_col=[0,1,2])
mats_info = pd.read_csv('../Demand/matrices_info.txt', delimiter=';', index_col=0)

C:\Users\haris.ballis\.conda\envs\py3\lib\site-packages\numpy\lib\arraysetops.py:395: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [45]:
df = mats.join(mats_info.NAME)
df.reset_index(inplace=True)
df = df.drop('MATRIXNO', 1)
df['NAME'] = df['NAME'].astype('category')
df = df.set_index(['NAME', 'FROMNO', 'TONO'])
df.sort_index(inplace=True)

In [46]:
demand = df.loc[['HBW_C', 'HBW_X', 'HBEDU_C', 'HBEDU_X', 'HBO_C', 'HBO_X', 'HBSH_C', 'HBSH_X', 'NHB_C', 'NHB_X',],:]

In [47]:
# Cleaning
demand.reset_index(inplace=True)
demand = demand.assign(Purpose = lambda x: x.NAME.str.split('_').str[0])
demand = demand.assign(Direction = lambda x: x.NAME.str.split('_').str[1])
demand.drop('NAME', axis=1, inplace=True)

demand.rename(columns={'FROMNO': 'From_Node', 'TONO': 'To_Node', 'VALUE': 'Trips'}, inplace=True)
demand.Purpose.astype('category')
demand.Direction.astype('category')

demand.set_index(['Purpose', 'Direction', 'From_Node', 'To_Node'], inplace=True)

### Test demand

In [48]:
demand_synthPop = demand.xs(['HBW', 'C'], level=[0,1], drop_level=False)

In [49]:
# Departure time
depTime = '08:00' # 8 hours
depTime_dev = '00:30' # deviation

In [50]:
# Create integer trips
# To-Do improve method
trips = demand_synthPop.Trips.round()

# exclude 0 trips
trips = trips[trips!=0]

pop = []
agent_id=1
for trip in trips.iteritems():
    idx = trip[0]
    val = trip[1]
    # Dictionary with the trips info
    d = dict(zip(trips.index.names, idx))
    for t in range(0, int(val)):
        
        # Create the agent
        new_agent = Agent(id=agent_id)
        
        # Create the plans
        # we have only one plan
        new_plan = Plan(selected='yes')        
        
        # Create the activities
        # Get Node's coords
        x, y = zone_centroids.loc[d['From_Node']]
        end_time = randNormal_time(depTime, depTime_dev)
                
        # To-Do must generate random points in the zone
        new_act = Activity(type=d['Purpose'], x=x, y=y, end_time=end_time)
        
        # Create the leg
        new_leg = Leg(mode='car')
        
        new_act.legs.append(new_leg)
        new_plan.activities.append(new_act)
        new_agent.plans.append(new_plan)
        pop.append(new_agent)
        agent_id+=1
        

In [51]:
xml = build_pop_xml(pop)

In [52]:
# Write the xml
ElementTree(xml).write('pop.xml')